In [1]:
# import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import cv2
import math
import torch
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import numpy as np
import matplotlib.colorbar as colorbar

import requests
from io import BytesIO
from PIL import Image
import numpy as np
from maskrcnn_benchmark.utils import cv2_util
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

from moviepy.editor import VideoFileClip

# this makes our figures bigger
pylab.rcParams['figure.figsize'] = 20*1.5, 12*1.5

from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.structures.keypoint import keypoints_to_heat_map
from maskrcnn_benchmark.modeling.roi_heads.keypoint_head.loss import project_keypoints_to_heatmap
from predictor import COCODemo

In [2]:
def load(img_path):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    
    pil_image = Image.open(img_path).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def load_video(video_path,frame_id=[0]):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    
    video_clip = VideoFileClip(video_path)
    if frame_id is None:
        nframes = int(video_clip.fps * video_clip.duration)
        frame_id = range(nframes)
    images = []
    for i in frame_id:
        image = video_clip.get_frame(i/video_clip.fps)
        # convert to BGR format
        image = np.array(image)[:, :, [2, 1, 0]]
        images += [image]
    video_clip.close()
    
    return images

def imshow(img):
    plt.gca().invert_yaxis()
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")
    
def sigmoid(x):
  return 1 / (1 + math.exp(-x))
# define vectorized sigmoid
sigmoid_v = np.vectorize(sigmoid)

# num keypoints per animal, heatmap
def calculate_peaks(numparts, heatmap_avg):
    score = np.ones((numparts,)) * 0.000001
    all_peaks = []
    peak_counter = 0
    if len(score) < numparts:
        score = score[:numparts]
        print("score", score)
        ##logger.ERROR(‘Not enough scores provided for number of parts’)
        # return
    # threshold_detection = params[‘thre1’]
    # tic_localmax=time.time()
    for part in range(numparts):
        map_ori = heatmap_avg[part, :, :]
        map = map_ori
        map_left = np.zeros(map.shape)
        map_left[1:, :] = map[:-1, :]
        map_right = np.zeros(map.shape)
        map_right[:-1, :] = map[1:, :]
        map_up = np.zeros(map.shape)
        map_up[:, 1:] = map[:, :-1]
        map_down = np.zeros(map.shape)
        map_down[:, :-1] = map[:, 1:]
        #peaks_binary = np.logical_and(np.logical_and(np.logical_and(map >= map_left, map >= map_right),
                                                     #np.logical_and(map >= map_up, map >= map_down)), map > score[part])
        peaks_binary = (sigmoid_v(map_ori) > .2)
        #print("pb shap", np.shape(peaks_binary))
        peaks = list(zip(np.nonzero(peaks_binary)[1], np.nonzero(peaks_binary)[0]))  # note reverse
        peaks_with_score_and_id = [x + (map_ori[x[1], x[0]], i + peak_counter,) for i, x in
                                   enumerate(peaks)]  # if x[0]>0 and x[1]>0 ]
        all_peaks.append(peaks_with_score_and_id)
        peak_counter += len(peaks)
    return all_peaks



In [3]:
params = {
    'text.latex.preamble': ['\\usepackage{gensymb}'],
    'image.origin': 'lower',
    'image.interpolation': 'nearest',
    'image.cmap': 'jet',
    'axes.grid': False,
    'savefig.dpi': 150,  # to adjust notebook inline plot size
    'axes.labelsize': 10, # fontsize for x and y labels (was 10)
    'axes.titlesize': 12,
    'font.size': 12, # was 10
    'legend.fontsize': 10, # was 10
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    #'text.usetex': True,
    'figure.figsize': [20, 12],
    'font.family': 'serif',
}
matplotlib.rcParams.update(params)

In [4]:
def overlay_keypoints(image, predictions):
        #keypoints = predictions.get_field("keypoints")
        kps = predictions#keypoints.keypoints
        scores = kps.new_ones((kps.size(0), kps.size(1)))
        kps = torch.cat((kps[:, :, 0:2], scores[:, :, None]), dim=2).numpy()
        for region in kps:
            kfun = BeeKeypoints
            image = vis_keypoints_others(
                    image,
                    region.transpose((1, 0)),
                    kp_thresh=0,
                    kfun=kfun)
                
        return image


In [5]:
import cv2
import torch
from torchvision import transforms as T

from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
from maskrcnn_benchmark.structures.image_list import to_image_list
from maskrcnn_benchmark.modeling.roi_heads.mask_head.inference import Masker
from maskrcnn_benchmark import layers as L
from maskrcnn_benchmark.utils import cv2_util

import numpy as np
import matplotlib.pyplot as plt
from maskrcnn_benchmark.structures.keypoint import PersonKeypoints, BeeKeypoints, FlyKeypoints


def vis_keypoints_others(img, kps, kp_thresh=2, alpha=0.7, kfun=PersonKeypoints):
    """Visualizes keypoints (adapted from vis_one_image).
    kps has shape (4, #keypoints) where 4 rows are (x, y, logit, prob).
    """
    img = img.copy()
    dataset_keypoints = kfun.NAMES
    kp_lines = kfun.CONNECTIONS

    # Convert from plt 0-1 RGBA colors to 0-255 BGR colors for opencv.
    cmap = plt.get_cmap('rainbow')
    colors = [cmap(i) for i in np.linspace(0, 1, len(kp_lines) + 2)]
    colors = [(c[2] * 255, c[1] * 255, c[0] * 255) for c in colors]

    # Perform the drawing on a copy of the image, to allow for blending.
    kp_mask = np.copy(img)
    
    # Draw the keypoints.
    for l in [0]:#range(len(kp_lines)):
        i1 = kp_lines[l][0]
        i2 = kp_lines[l][1]
        p1 = kps[0, i1], kps[1, i1]
        p2 = kps[0, i2], kps[1, i2]
        #if kps[2, i1] > kp_thresh and kps[2, i2] > kp_thresh:
         #   cv2.line(
          #      kp_mask, p1, p2,
           #     color=colors[l], thickness=2, lineType=cv2.LINE_AA)
        if kps[2, i1] > kp_thresh:
            cv2.circle(
                kp_mask, p1,
                radius=16, color=colors[l], thickness=-1, lineType=cv2.LINE_AA)
        #if kps[2, i2] > kp_thresh:
         #   cv2.circle(
          #      kp_mask, p2,
           #     radius=16, color=colors[l], thickness=-1, lineType=cv2.LINE_AA)

    # Blend the keypoints.
    return cv2.addWeighted(img, 1.0 - alpha, kp_mask, alpha, 0)

def bounding_box(points):
    x_coordinates, y_coordinates = zip(*points)
    return [min(x_coordinates), min(y_coordinates), max(x_coordinates), max(y_coordinates)]

def get_centroid(coord):
    return [(coord[0]+coord[2])/2, (coord[1]+coord[3])/2]

def get_dist(p1, p2):
    dist = math.hypot(p2[0] - p1[0], p2[1] - p1[1])
    return dist


In [6]:
#test_file = 'bee_test.png'
#image = load(test_file)
base = "../tools/datasets/bee/validation/"
#base = "../tools/datasets/bee/train/"
base_val = "../tools/datasets/bee/annotations/validation.json"
#base_val = "../tools/datasets/bee/annotations/train_bee_annotations2018_nondup.json"

import json
with open(base_val) as f:
    data_an = json.load(f)
#print(data_an["annotations"][0])

test_files = []
for p in data_an['images'][:]:
    test_files.append(str(p['file_name']))

print(test_files)
print(len(test_files))
test_images = []
for file in test_files:
    test_images.append(load(base + file))



['000000051501.jpg', '000000051602.jpg', '000000051656.jpg', '000000051770.jpg', '000000052209.jpg', '000000052239.jpg', '000000052248.jpg', '000000052271.jpg', '000000052328.jpg', '000000052361.jpg', '000000052413.jpg', '000000052563.jpg', '000000052583.jpg', '000000052587.jpg', '000000052738.jpg', '000000052799.jpg', '000000052861.jpg', '000000053184.jpg', '000000053247.jpg', '000000053341.jpg', '000000053573.jpg', '000000053614.jpg', '000000053776.jpg', '000000053802.jpg', '000000053827.jpg', '000000053842.jpg', '000000053857.jpg', '000000053866.jpg', '000000053890.jpg', '000000053916.jpg']
30


In [135]:
s_v1 = []
s_v2 = []
s_v3 = []

c_v1 = []
c_v2 = []
c_v3 = []

u_v1 = []
u_v2 = []
u_v3 = []

In [181]:
#single mse dp
import matplotlib.gridspec as gridspec
from itertools import chain

data_size = "135_new"
#its = range(3000,5001,1000)
#its = range(1000,10001,1000)

its = range(1000,4001,1000)
its = range(1000,6001,1000)
#its = range(6000,8001,1000)
b = "23.1"
v = "2"
losses = ["p32combined_res.001/3", "p42/3", ]
#losses = ["p2combined_res/3","p3p2fcombined_res/3"]
#losses = ["standard_res/3", "p3fstandard_res/3"]
loss = losses[1]
#t = []
#its = [1000]
for it in its:
    t = []
    print(it,"it")
    if it < 1000:
        it = "0000" + str(it)
    elif it > 9999:
        it = "00" + str(it)
    else:
        it = "000" + str(it)
    config_file = "/home/bsb2144/directpose/configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml"

    cfg.merge_from_file("/home/bsb2144/directpose/configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml")
    cfg.merge_from_list(['DATALOADER.NUM_WORKERS', '2', \
                         'DATATYPE', 'bee', \
                         'OUTPUT_DIR', 'training_dir/' + data_size + '/' + loss +'/fcos_kps_ms_training_R_50_FPN_1x_bee1', \
                         'MODEL.WEIGHT', '/home/bsb2144/directpose/tools/training_dir/' + data_size + '/'+ loss +'/fcos_kps_ms_training_R_50_FPN_1x_bee/model_'+it+'.pth',\
                         'DATASETS.TEST', "('bee_val_cocostyle',)",\
                         'DATASETS.TRAIN', "('bee_train_cocostyle', )",\
                        ])

    coco_demo = COCODemo(
                cfg,
                min_image_size=800,
                confidence_threshold=0.2,
    )


    for idx, image in enumerate(test_images):#test_images):
        mse_per_bee = []
        predictions, tps, hm_res, stride, hms, targets, c_loss = coco_demo.run_on_opencv_image(image)
        t.append(tps)
    u_v3.append(t)
        
        
        
        
        
        

1000 it
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=100, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=21, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=85, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=14, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=55, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(

TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=100, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=19, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=62, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=18, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=53, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxe

TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=90, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=18, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=100, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=14, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=76, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxe

TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=94, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=18, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=87, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=15, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=100, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxe

TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=77, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=14, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=82, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=16, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=63, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes

TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=100, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=21, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=64, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=14, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=44, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxe

TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=97, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=19, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=56, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=18, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=43, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes

TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=100, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=18, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=100, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_boxes=14, image_width=2560, image_height=1440, mode=xyxy)
TARGETS:  None
feats:  5
feats:  1
feats:  256
SHAPE THING torch.Size([1, 10, 100, 180])
SHAPE THING torch.Size([1, 10, 50, 90])
SHAPE THING torch.Size([1, 10, 25, 45])
props grcnn
FIELDS:
predictions BoxList(num_boxes=81, image_width=2560, image_height=1440, mode=xyxy)
top preds BoxList(num_box

In [182]:
print(len(u_v3))

10


In [183]:
saver = {"standard":[{'v1':s_v1}, {'v2':s_v2}, {'v3':s_v3}],
"combined":[{'v1':c_v1}, {'v2':c_v2}, {'v3':c_v3}],
"unlabeled":[{'v1':u_v1}, {'v2':u_v2}, {'v3':u_v3}]}


In [184]:
print(saver['unlabeled'][2]['v3'][0])

[BoxList(num_boxes=21, image_width=2560, image_height=1440, mode=xyxy), BoxList(num_boxes=14, image_width=2560, image_height=1440, mode=xyxy), BoxList(num_boxes=10, image_width=2560, image_height=1440, mode=xyxy), BoxList(num_boxes=12, image_width=2560, image_height=1440, mode=xyxy), BoxList(num_boxes=12, image_width=2560, image_height=1440, mode=xyxy), BoxList(num_boxes=12, image_width=2560, image_height=1440, mode=xyxy), BoxList(num_boxes=15, image_width=2560, image_height=1440, mode=xyxy), BoxList(num_boxes=16, image_width=2560, image_height=1440, mode=xyxy), BoxList(num_boxes=16, image_width=2560, image_height=1440, mode=xyxy), BoxList(num_boxes=19, image_width=2560, image_height=1440, mode=xyxy), BoxList(num_boxes=10, image_width=2560, image_height=1440, mode=xyxy), BoxList(num_boxes=12, image_width=2560, image_height=1440, mode=xyxy), BoxList(num_boxes=17, image_width=2560, image_height=1440, mode=xyxy), BoxList(num_boxes=14, image_width=2560, image_height=1440, mode=xyxy), BoxLi

In [185]:
#out_dir = "tar.json" 
#import pickle
#with open('all_135_test_boxes.pkl', 'wb') as f:
 #   pickle.dump(saver, f)

In [2]:
import pickle
with open('all_5_test_boxes.pkl', 'rb') as f:
    r5 = pickle.load(f)

In [7]:
print((r5['standard'][0]['v1'][0][10].get_field('keypoints').keypoints))

tensor([[[1.7145e+03, 1.6383e+02, 1.0000e+00],
         [1.6954e+03, 3.8389e+02, 1.0000e+00],
         [1.6898e+03, 2.9595e+02, 1.0000e+00],
         [1.6664e+03, 3.9520e+02, 1.0000e+00],
         [1.7325e+03, 4.0045e+02, 1.0000e+00]],

        [[4.8767e+02, 5.0381e+02, 1.0000e+00],
         [4.3763e+02, 2.6752e+02, 1.0000e+00],
         [4.5817e+02, 3.5960e+02, 1.0000e+00],
         [4.6196e+02, 2.2315e+02, 1.0000e+00],
         [3.9087e+02, 2.4037e+02, 1.0000e+00]],

        [[8.7800e+02, 9.3290e+02, 1.0000e+00],
         [1.0348e+03, 1.0828e+03, 1.0000e+00],
         [9.6323e+02, 1.0232e+03, 1.0000e+00],
         [1.0220e+03, 1.1214e+03, 1.0000e+00],
         [1.0813e+03, 1.0785e+03, 1.0000e+00]],

        [[1.2485e+03, 4.8405e+02, 1.0000e+00],
         [1.1447e+03, 2.0603e+02, 1.0000e+00],
         [1.1838e+03, 3.1508e+02, 1.0000e+00],
         [1.1676e+03, 1.4840e+02, 1.0000e+00],
         [1.0803e+03, 1.8257e+02, 1.0000e+00]],

        [[6.9586e+02, 7.3726e+02, 1.0000e+00],
     

In [3]:
from maskrcnn_benchmark.data.datasets.evaluation import evaluate
from maskrcnn_benchmark.data import make_data_loader
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.structures.keypoint import keypoints_to_heat_map
from maskrcnn_benchmark.modeling.roi_heads.keypoint_head.loss import project_keypoints_to_heatmap
from predictor import COCODemo


config_file = "/home/bsb2144/directpose/configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml"

cfg.merge_from_file("/home/bsb2144/directpose/configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml")
cfg.merge_from_list(['DATALOADER.NUM_WORKERS', '2', \
                     'DATATYPE', 'bee', \
                     'OUTPUT_DIR', 'training_dir/fcos_kps_ms_training_R_50_FPN_1x_bee1', \
                     'MODEL.WEIGHT', '/home/bsb2144/directpose/tools/training_dir/fcos_kps_ms_training_R_50_FPN_1x_bee/mode.pth',\
                     'DATASETS.TEST', "('bee_val_cocostyle',)",\
                     'DATASETS.TRAIN', "('bee_train_cocostyle', )",\
                    ])

ds = make_data_loader(cfg)

#res = evaluate()

train san check: True
DATA Path;  ('bee_val_cocostyle',)
name?
/home/bsb2144/directpose/tools/datasets/bee/annotations/validation.json
datasets/bee/validation
BeeKeypoints
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
len ids pre  30


In [4]:
res = evaluate(ds, r5['standard'][0]['v1'][10], 'pathhe')

Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
coco_gt <pycocotools.coco.COCO object at 0x2b2cfad2ab70>
coco_dt <pycocotools.coco.COCO object at 0x2b2cf98cdd30>
iou_type keypoints
datatype bee
Running per image evaluation...
Evaluate annotation type *keypoints*
sigmas [0.026 0.025 0.025 0.035 0.035 0.079 0.079 0.072 0.072 0.062 0.062 0.107
 0.107 0.087 0.087 0.089 0.089]
g shape 15 [699 277   1 570 505   1 611 452   1 572 572   1 493 491   1]
xg shape 5 [699 570 611 572 493]
yg shape 5 [277 505 452 572 491]
vg shape 5 [1 1 1 1 1]
xd [1252.87573242 1478.08410645 1408.14257812 1489.4128418  1533.1105957 ] 5
yd [ 69.79717255 196.49633789 161.53289795 235.11430359 160.48620605] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 27942.308888135925
xd [1110.81591797  870.95996094  948.97888184  826.48461914  834.48284912] 5
yd [274.9200

xd [1110.81591797  870.95996094  948.97888184  826.48461914  834.48284912] 5
yd [274.92001343 270.03131104 263.32925415 239.48184204 327.59320068] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 23033.321883740522
xd [1743.84436035 1581.68591309 1634.62194824 1508.00109863 1567.59716797] 5
yd [285.27108765 465.99481201 423.26879883 473.13146973 522.33966064] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 23033.321883740522
xd [738.94146729 739.82867432 738.69012451 790.7401123  688.59075928] 5
yd [834.95220947 611.79949951 693.20916748 564.01281738 565.04998779] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 2

xd [ 984.92504883 1142.4552002  1081.5949707  1137.7232666  1212.2166748 ] 5
yd [ 881.04040527 1090.05029297 1031.5222168  1142.8190918  1099.84558105] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 24188.50346755665
xd [1178.90881348 1186.20874023 1181.28015137 1192.17700195 1184.31713867] 5
yd [653.91571045 937.10430908 858.70147705 987.25494385 981.67895508] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 24188.50346755665
xd [2271.88525391 2256.16162109 2236.64306641 2303.8059082  2258.12548828] 5
yd [359.60424805  98.90818024 160.59028625 120.51509857  86.13070679] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]

xd [1743.84436035 1581.68591309 1634.62194824 1508.00109863 1567.59716797] 5
yd [285.27108765 465.99481201 423.26879883 473.13146973 522.33966064] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 26857.507330353652
xd [738.94146729 739.82867432 738.69012451 790.7401123  688.59075928] 5
yd [834.95220947 611.79949951 693.20916748 564.01281738 565.04998779] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 26857.507330353652
xd [365.1880188  438.33947754 406.71960449 423.24804688 428.62481689] 5
yd [ 888.91888428 1135.41821289 1046.71020508 1169.90905762 1158.57702637] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 2

xd [644.70428467 575.6920166  588.16882324 584.75189209 543.90637207] 5
yd [124.96644592 -28.78897858   4.42296505 -31.11950111 -26.78200912] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 26275.29632563637
xd [793.90917969 888.49700928 853.75439453 874.6116333  910.44934082] 5
yd [ -1.48848259 177.30787659 111.58466339 232.88973999 206.10871887] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 26275.29632563637
xd [1836.24230957 1792.10119629 1842.49682617 1821.57543945 1803.4654541 ] 5
yd [1169.31115723 1236.17431641 1219.7253418  1236.2166748  1242.21557617] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 262

...

 0.031684]
other 26233.060820270286
xd [1723.63647461 1784.78051758 1774.05529785 1837.45153809 1743.40588379] 5
yd [661.72753906 402.51971436 467.65539551 378.89053345 358.20944214] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 26233.060820270286
xd [2015.31994629 1931.18408203 1972.51733398 1946.97668457 1858.26806641] 5
yd [1030.95996094  794.36315918  882.10400391  758.39489746  791.33978271] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 26233.060820270286
xd [498.01171875 479.86132812 490.3788147  444.42190552 503.99441528] 5
yd [ 872.02197266 1107.07507324 1024.7220459  1133.71008301 1142.43762207] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796

xd [1724.65112305 1732.73657227 1728.16943359 1783.13122559 1687.04211426] 5
yd [956.08306885 712.72540283 784.94616699 670.83953857 675.61743164] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 24839.528437552915
xd [866.83953857 927.93292236 892.68457031 907.52996826 974.27502441] 5
yd [1257.29992676 1066.25354004 1173.73583984 1025.64916992 1028.60559082] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 24839.528437552915
xd [1699.58947754 1784.07470703 1763.38513184 1840.20910645 1745.56884766] 5
yd [651.33782959 418.44561768 485.71011353 391.9095459  363.57937622] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
ot

xd [661.60839844 688.82470703 680.83691406 661.40374756 710.5994873 ] 5
yd [-39.7494812  115.34424591  33.44064331 161.02798462 158.07846069] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 23860.850362047033
xd [1392.38256836 1484.61206055 1434.89074707 1533.96801758 1442.03942871] 5
yd [ 771.97650146 1030.60852051  935.65283203 1075.09399414 1052.49536133] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 23860.850362047033
xd [1724.65112305 1732.73657227 1728.16943359 1783.13122559 1687.04211426] 5
yd [956.08306885 712.72540283 784.94616699 670.83953857 675.61743164] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
ot

 0.031684]
other 24998.084106587048
xd [1147.95385742 1161.38598633 1153.40722656 1210.3807373  1120.67565918] 5
yd [793.32720947 547.89782715 640.83233643 508.42770386 505.06045532] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 24998.084106587048
xd [527.42584229 617.53686523 575.8034668  624.02368164 601.32232666] 5
yd [187.63528442 429.55700684 339.33862305 467.25741577 448.82910156] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 24998.084106587048
xd [1438.98791504 1274.6809082  1324.38452148 1284.89831543 1216.25085449] 5
yd [539.90356445 359.39260864 418.33914185 319.8956604  365.2074585 ] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 

yd [1029.81164551  801.77587891  886.33679199  796.28210449  775.56860352] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 28224.763063664504
xd [1462.48254395 1362.54040527 1394.8314209  1381.87792969 1313.42407227] 5
yd [834.87091064 632.07141113 706.50842285 606.38696289 624.91784668] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 28224.763063664504
xd [603.42370605 591.85009766 603.90209961 585.66827393 603.85046387] 5
yd [438.75057983 747.83691406 665.97674561 803.36077881 800.63214111] 5
test vars [0.002704 0.0025   0.0025   0.0049   0.0049   0.024964 0.024964 0.020736
 0.020736 0.015376 0.015376 0.045796 0.045796 0.030276 0.030276 0.031684
 0.031684]
other 28224.763063664504
g shape 15 [600 466   1 593 743   1 599 646   1 644 79

In [15]:
aps = (res[0])

In [ ]:
OrderedDict([('keypoints',
              OrderedDict([('AP', 0.779651305579224), 
                           ('AP50', 0.8200120850549557)
                                        , ('AP75', 0.8029049611437388), ('APm', -1.0), ('APl', 0.7892891567924767)]))])

In [24]:
print((aps.results['keypoints']['AP']))

0.779651305579224
